In [169]:
import pandas as pd
import numpy as np
!pip install pandas-ta
import pandas_ta as ta
from tensorflow.keras.layers import LSTM,Dense,Dropout,BatchNormalization,Bidirectional,Input,Conv1D,Flatten,Conv1D
from tensorflow.keras.activations import relu,sigmoid,tanh,linear
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
# from keras_tuner.tuners import BayesianOptimization
# from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Dropout, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
# from keras_tuner.tuners import BayesianOptimization
from tensorflow.keras.callbacks import EarlyStopping
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split



In [170]:
df = pd.read_csv('/content/CBOE_DLY_SPX, 60 (1).csv')

In [171]:
df = df.drop(columns='Volume',axis=1)

In [172]:
# import pandas as pd
df['time'] = pd.to_datetime(df['time'])
# df['week'] = df['time'].dt.day_name()
df.set_index('time',inplace=True)
# print(df['week'].unique)

<ipython-input-172-57b6a2d72a96>:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['time'] = pd.to_datetime(df['time'])


In [173]:
df['next_hour_close'] = df['close'].shift(-1)
df['ema5'] = ta.ema(df['close'] , length=5)
df['ema10'] = ta.ema(df['close'] , length=10)
df['ema20'] = ta.ema(df['close'] , length=20)
df['ema50'] = ta.ema(df['close'] , length=50)
df['ema200'] = ta.ema(df['close'] , length=200)
df['atr'] = df['high'] - df['low']
df['atr'] = df['atr'].rolling(14).mean()
df['rsi'] = ta.rsi(df['close'] , length=14)
macd = ta.macd(df['close'], fast=12, slow=26, signal=9)
df = pd.concat([df,macd],axis=1)
df['pivot'] = (df['high'] + df['low'] + df['close']) / 3
df['support_1'] = (2 * df['pivot']) - df['high']
df['support_2'] = df['pivot'] - (df['high'] - df['low'])
df['resistance_1'] = (2 * df['pivot']) - df['low']
df['resistance_2'] = df['pivot'] + (df['high'] - df['low'])
bollinger = ta.bbands(df['close'],length=20,std=2)
df = pd.concat([df,bollinger],axis=1)
stochastic = ta.stoch(df['high'], df['low'], df['close'], k=14, d=3, smooth_k=3)
df = pd.concat([df, stochastic], axis=1)
adx = ta.adx(df['high'], df['low'], df['close'], length=14)
df = pd.concat([df, adx], axis=1)
def fibonacci_retracement(df):
    high = df['high'].max()
    low = df['low'].min()

    diff = high - low

    df['fibonaccilevel2'] = high - 0.382 * diff
    df['fibonaccilevel3'] = high - 0.5 * diff
    df['fibonaccilevel4'] = high - 0.618 * diff
    df['fibonaccilevel5'] = high - 0.764 * diff

    return df

# fibonacci_retracement(df)

def stochastic_oscillator(df, period=14, ma_period=3):
    df['L14'] = df['low'].rolling(window=period).min()
    df['H14'] = df['high'].rolling(window=period).max()
    df['%K'] = (df['close'] - df['L14']) * 100 / (df['H14'] - df['L14'])
    df['%D'] = df['%K'].rolling(window=ma_period).mean()
    return df
stochastic_oscillator(df)

# Additional pivot-based levels
df['support_3'] = df['low'] - 2 * (df['high'] - df['pivot'])
df['resistance_3'] = df['high'] + 2 * (df['pivot'] - df['low'])
# Fibonacci levels (common ratios)
fib_range = df['high'] - df['low']
df['fib_236'] = df['high'] - fib_range * 0.236
df['fib_382'] = df['close'] + fib_range * 0.382  # Common retracement
df['fib_618'] = df['close'] - fib_range * 0.618  # Golden ratio
# Woodie's variation (uses 2*close in calculation)
df['woodie_pivot'] = (df['high'] + df['low'] + 2 * df['close']) / 4
df['woodie_s1'] = 2 * df['woodie_pivot'] - df['high']
df['woodie_r1'] = 2 * df['woodie_pivot'] - df['low']
# Camarilla system (uses different multipliers)
df['camarilla_s1'] = df['close'] - (df['high'] - df['low']) * 1.1 / 4
df['camarilla_r1'] = df['close'] + (df['high'] - df['low']) * 1.1 / 4
# Midpoints between key levels
df['mid_support'] = (df['support_1'] + df['support_2']) / 2
df['mid_resistance'] = (df['resistance_1'] + df['resistance_2']) / 2
# Alternative pivot calculation (includes open price)
df['alt_pivot'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
# Moving average of pivot points
df['pivot_ma7'] = df['pivot'].rolling(window=7).mean()
df['pivot_ma21'] = df['pivot'].rolling(window=21).mean()
# Price range calculations
df['daily_range'] = df['high'] - df['low']
df['range_pct'] = (df['daily_range'] / df['close']) * 100

df['HA_close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
df['HA_open'] = (df['open'].shift(1) + df['close'].shift(1)) / 2
df['HA_high'] = df[['high', 'HA_open', 'HA_close']].max(axis=1)
df['HA_low'] = df[['low', 'HA_open', 'HA_close']].min(axis=1)
df['HA_open'].iloc[0] = (df['open'].iloc[0] + df['close'].iloc[0]) / 2


<ipython-input-173-764a1d8ba575>:77: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['HA_open'].iloc[0] = (df['open'].iloc[0] + df['close'].iloc[0]) / 2


In [174]:
df.dropna(inplace=True)

In [175]:
features = [#'open',
#  'high',
#  'low',
   'close',
    #  'ema5',
    # 'ema10',
  #  'ema20',
  #  'HA_close',
    # 'HA_open',
  #  'HA_high',
    # 'HA_low',
  # 'ema50',
  # 'ema200',
  #  'atr',
  # 'rsi',
  # 'MACD_12_26_9',
  # 'MACDh_12_26_9',
  # 'MACDs_12_26_9',
      # 'pivot',
    # 'fib_236',
  #  'support_1',
      # 'support_2',
    #  'resistance_1',
        # 'resistance_2']
 # 'BBL_20_2.0',
 # # 'BBM_20_2.0',
 # 'BBU_20_2.0',
 # # 'BBB_20_2.0',
 # # 'BBP_20_2.0',
 # # 'STOCHk_14_3_3',
 # # 'STOCHd_14_3_3',
  # 'ADX_14',
 # # 'DMP_14',
 # # 'DMN_14',
 # 'L14',
 # 'H14',
 # # '%K',
 # # '%D',
  #  'support_3',
  #  'resistance_3']
        #  'fib_382',
    # 'fib_618']
    #  'woodie_pivot']
#  'woodie_s1',
  #  'woodie_r1']
          #  'camarilla_s1',
         'camarilla_r1']
    # 'mid_support',
    # 'mid_resistance']
 # 'alt_pivot',
  # 'pivot_ma7']
 # 'pivot_ma21']
 # # 'daily_range',
 # 'range_pct']
target = ['next_hour_close']

# close              0.999833
# camarilla_s1       0.999827
# camarilla_r1       0.999825
# woodie_pivot       0.999824
# fib_382            0.999819
# pivot              0.999818
# woodie_s1          0.999813
# support_1          0.999810
# woodie_r1          0.999809
# resistance_1       0.999804
# alt_pivot          0.999803
# fib_618            0.999802
# low                0.999782
# mid_support        0.999779
# high               0.999773
# mid_resistance     0.999770
# support_2          0.999737
# resistance_2       0.999722
# open               0.999720
# ema5               0.999698
# support_3          0.999639
# resistance_3       0.999621
# ema10              0.999494
# pivot_ma7          0.999487
# ema20              0.999074
# L14                0.998902
# BBM_20_2.0         0.998797
# H14                0.998785
# pivot_ma21         0.998697
# BBL_20_2.0         0.998270
# BBU_20_2.0         0.998088
# ema50              0.997838
# ema200             0.992243

In [176]:
X = df[features].values
y = df[target].values.reshape(-1, 1)

In [177]:
x_scaler = StandardScaler()
y_scaler = StandardScaler()


X_scaled = x_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

In [178]:
Sequence_len = 40

def creat_sequences(X,y,sequence_len):
  X_seq = []
  y_seq = []
  for i in range(len(X)-sequence_len):
    X_seq.append(X[i:i+sequence_len])
    y_seq.append(y[i+sequence_len])
  return np.array(X_seq),np.array(y_seq)

In [179]:
X_final,y_final = creat_sequences(X_scaled,y_scaled,Sequence_len)

In [180]:
print(X_final.shape)
print(y_final.shape)

(10378, 40, 2)
(10378, 1)


In [181]:
split_index = int(len(X_final)*0.90)
x_train = X_final[:split_index]
y_train = y_final[:split_index]
x_test = X_final[split_index:]
y_test = y_final[split_index:]

In [182]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(9340, 40, 2)
(9340, 1)
(1038, 40, 2)
(1038, 1)


In [183]:
def objective(trial):
    # هایپرپارامترهای پایه
    n_lstm_layers = trial.suggest_int("n_lstm_layers", 1, 3)
    n_dense_layers = trial.suggest_int("n_dense_layers", 1, 3)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # تعریف پارامترهای مجزا برای هر لایه LSTM
    lstm_units_list = [
        trial.suggest_int(f"lstm_units_{i}", 32, 512, step=32)
        for i in range(3)  # حداکثر 3 لایه
    ]
    lstm_dropout = [
        trial.suggest_float(f"lstm_dropout_{i}", 0.0, 0.5, step=0.1)
        for i in range(3)
    ]
    lstm_l2 = [
        trial.suggest_float(f"lstm_l2_{i}", 1e-6, 1e-2, log=True)
        for i in range(3)
    ]

    # تعریف پارامترهای مجزا برای هر لایه Dense
    dense_units_list = [
        trial.suggest_int(f"dense_units_{i}", 16, 256, step=16)
        for i in range(3)  # حداکثر 3 لایه
    ]
    dense_dropout = [
        trial.suggest_float(f"dense_dropout_{i}", 0.0, 0.5, step=0.1)
        for i in range(3)
    ]
    dense_l2 = [
        trial.suggest_float(f"dense_l2_{i}", 1e-6, 1e-2, log=True)
        for i in range(3)
    ]

    # ساخت مدل
    model = Sequential()

    # اولین لایه LSTM
    model.add(Bidirectional(
        LSTM(
            lstm_units_list[0],
            return_sequences=(n_lstm_layers > 1),
            kernel_regularizer=l2(lstm_l2[0])  # L2 مجزا برای لایه اول
        ),
        input_shape=(Sequence_len, x_train.shape[2])
    ))
    model.add(Dropout(lstm_dropout[0]))  # Dropout مجزا برای لایه اول

    # سایر لایههای LSTM
    for i in range(1, n_lstm_layers):
        return_sequences = (i < n_lstm_layers - 1)
        model.add(Bidirectional(
            LSTM(
                lstm_units_list[i],
                return_sequences=return_sequences,
                kernel_regularizer=l2(lstm_l2[i])  # L2 مجزا
            )
        ))
        model.add(Dropout(lstm_dropout[i]))  # Dropout مجزا

    # لایههای Dense
    for i in range(n_dense_layers):
        model.add(Dense(
            dense_units_list[i],
            activation='relu',
            kernel_regularizer=l2(dense_l2[i])  # L2 مجزا برای Dense
        ))
        model.add(Dropout(dense_dropout[i]))  # Dropout مجزا

    # لایه خروجی
    model.add(Dense(1, activation='linear'))

    # کامپایل مدل
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )
    Es = EarlyStopping(monitor='val_mae',patience=10)


    # آموزش مدل
    history = model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        batch_size=batch_size,
        epochs=30,
        verbose=1,
        callbacks=[Es]
    )

    return min(history.history["val_loss"])

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                        │ (None, 384)                 │         594,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 48)                  │          18,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │              49 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 612,961 (2.34 MB)

 Trainable params: 612,961 (2.34 MB)

 Non-trainable params: 0 (0.00 B)

In [184]:
study = optuna.create_study(direction="minimize")  # هدف کاهش loss است
study.optimize(objective, n_trials=50)  # تعداد اجراهای مختلف

Epoch 1/200
292/292 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0268 - mae: 0.0711 - val_loss: 0.0014 - val_mae: 0.0231
Epoch 2/200
292/292 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - loss: 0.0013 - mae: 0.0234 - val_loss: 0.0013 - val_mae: 0.0231
Epoch 3/200
292/292 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 0.0013 - mae: 0.0231 - val_loss: 0.0012 - val_mae: 0.0216
Epoch 4/200
292/292 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0012 - mae: 0.0228 - val_loss: 0.0011 - val_mae: 0.0198
Epoch 5/200
292/292 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0011 - mae: 0.0214 - val_loss: 0.0012 - val_mae: 0.0220
Epoch 6/200
292/292 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0011 - mae: 0.0220 - val_loss: 0.0010 - val_mae: 0.0197
Epoch 7/200
292/292 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0010 - mae: 0.0209 - val_loss: 9.6389e-04 - val_mae: 0.0193
Epoch 8/200
292/292 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0010 - mae: 0.0208 - val_loss: 9.4702e-04 - val_mae: 0.0189
Epoch 9/200
292/292 ━━━━━━━━━━━

In [185]:
print("Best hyperparameters: ", study.best_params)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 9.3549e-04 - mae: 0.0195
Test Loss: 0.0010328423231840134, Test MAE: 0.02050572820007801
